<a href="https://colab.research.google.com/github/SumaSri0915/Sentiment_Analysis/blob/main/LSTM%2BGLOVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Download GloVe Embeddings (for example, 300-dimensional embeddings)
!wget http://nlp.stanford.edu/data/glove.6B.zip  # Download the zip file

# Step 2: Unzip the file
!unzip glove.6B.zip

# Step 3: Load the GloVe embeddings
import numpy as np

def load_glove_embeddings(glove_file_path):
    embeddings_index = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Specify the path to the downloaded GloVe file (300d embeddings)
glove_file_path = '/content/glove.6B.300d.txt'  # Path to the GloVe file (300 dimensions)
embeddings_index = load_glove_embeddings(glove_file_path)

# Example: Print a sample word embedding
print('Sample embedding for "the":', embeddings_index.get('the'))

--2025-08-10 08:07:55--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-08-10 08:07:55--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-08-10 08:07:55--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [8]:
import numpy as np
import pandas as pd
import nltk
import string  # Import the string module here
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Download NLTK stopwords and punkt tokenizer
nltk.download('stopwords')
nltk.download('punkt')

# Load the IMDB dataset (replace with your actual path)
df = pd.read_csv('/content/IMDB-Dataset.csv')  # Update this with your correct file path

# Display the first few rows of the dataset
print(df.head())

# Clean the text function
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])  # Now string.punctuation will work
    # Tokenize the text
    words = nltk.word_tokenize(text)
    # Remove stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)
nltk.download('punkt_tab')
# Apply cleaning function to the 'review' column
df['cleaned_review'] = df['review'].apply(clean_text)

# Tokenize the text and convert to sequences
tokenizer = Tokenizer(num_words=5000)  # Set maximum number of words to 5000
tokenizer.fit_on_texts(df['cleaned_review'])
X = tokenizer.texts_to_sequences(df['cleaned_review'])

# Pad the sequences to ensure equal length
max_sequence_length = 200  # Adjust the sequence length as needed
X = pad_sequences(X, maxlen=max_sequence_length)

# Encode the labels (sentiment)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['sentiment'])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the GloVe embeddings
def load_glove_embeddings(glove_file_path):
    embeddings_index = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Update the path to the location where you've saved the GloVe file
glove_file_path = '/content/glove.6B.300d.txt'  # Update with the correct path
embeddings_index = load_glove_embeddings(glove_file_path)

# Create an embedding matrix using GloVe
embedding_dim = 300  # As we're using the 300d GloVe embeddings
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

for word, i in tokenizer.word_index.items():
    if i >= 5000:  # Only use top 5000 words
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Build the model using GloVe embeddings as pre-trained weights
model = Sequential()

# Embedding layer with pre-trained GloVe embeddings
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_sequence_length,
                    trainable=False))  # Freezing the embeddings

# Bidirectional LSTM layer
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))
from tensorflow.keras.optimizers import RMSprop

# Define learning rate
learning_rate = 0.001  # You can adjust this value

# Compile the model
model.compile(optimizer=RMSprop(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=8, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5)

# Print the classification report and accuracy
print("LSTM + GloVe Model Accuracy:", accuracy_score(y_test, y_pred))
print("LSTM + GloVe Classification Report:\n", classification_report(y_test, y_pred))

# Function to predict sentiment for new reviews
def predict_sentiment(review):
    review = clean_text(review)
    review_sequence = tokenizer.texts_to_sequences([review])
    review_padded = pad_sequences(review_sequence, maxlen=max_sequence_length)
    prediction = model.predict(review_padded)
    return "Positive" if prediction > 0.5 else "Negative"

# Example reviews for sentiment prediction
example_review_1 = "The movie was absolutely amazing, one of the best I've seen!"
example_review_2 = "It was a terrible movie. Boring and a waste of time."

# Get sentiment prediction for the reviews
print(predict_sentiment(example_review_1))
print(predict_sentiment(example_review_2))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │    54,378,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,378,000 (207.44 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 54,378,000 (207.44 MB)

Epoch 1/8
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 29s 21ms/step - accuracy: 0.7678 - loss: 0.4818 - val_accuracy: 0.8650 - val_loss: 0.3205
Epoch 2/8
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 29s 23ms/step - accuracy: 0.8655 - loss: 0.3219 - val_accuracy: 0.8721 - val_loss: 0.3021
Epoch 3/8
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 40s 22ms/step - accuracy: 0.8844 - loss: 0.2760 - val_accuracy: 0.8580 - val_loss: 0.3185
Epoch 4/8
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.8935 - loss: 0.2594 - val_accuracy: 0.8940 - val_loss: 0.2623
Epoch 5/8
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 39s 21ms/step - accuracy: 0.9063 - loss: 0.2301 - val_accuracy: 0.8925 - val_loss: 0.2615
Epoch 6/8
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.9198 - loss: 0.1967 - val_accuracy: 0.8891 - val_loss: 0.2677
Epoch 7/8
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.9354 - loss: 0.1679 - val_accuracy: 0.8917 - val_loss: 0.2691
Epoch 8/8
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.9462 - loss: 0